# 3. Assigning of urban contexts to inscriptions

*AIM*: Assign each inscription an urban context, depending on the available spatio-temporal data.

References: 

1) `Cities: Hanson J. W., An urban geography of the Roman world, 100 BC to AD 300. Oxford: Archaeopress; 2016. http://oxrep.classics.ox.ac.uk/oxrep/docs/Hanson2016/Hanson2016_Cities_OxREP.csv`

2) `Population estimates: Hanson J. W, Ortman S. G., A systematic method for estimating the populations of Greek and Roman settlements. J Roman Archaeol. 2017;30: 301–324.`


This script was originally published by `Kaše V, Heřmánková P, Sobotková A (2022) Division of labor, specialization and diversity in the ancient Roman cities: A quantitative approach to Latin epigraphy. PLoS ONE 17(6): e0269869. https://doi.org/10.1371/journal.pone.0269869` under a CC BY-SA 4.0 International License. 

https://github.com/sdam-au/social_diversity

The *Past Social Networks Project* adapted the script to fit the needs of the project research agenda.

# Requirements

In [21]:
import re
import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from pyproj import Geod
from shapely.geometry import LineString

import seaborn as sns
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk


In [22]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:
s = sddk.cloudSession("sciencedata.dk", ".", "648560@au.dk")
figpath = "./Petra_credentials/"
PSNP_figs = "./PSNP_shared/lat_socdiv/figures"

Your ScienceData username (e.g. '123456@au.dk'):  648560@au.dk
Your ScienceData password:  ········


connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/./


In [ ]:
# global parameters for plots
plt.style.use("fast")
plt.rcParams['figure.dpi'] = 300
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['font.size'] = 8

def save_figure(fig, fig_name):
    fig.savefig("../../figures/" + fig_name + ".eps") # saves locally
    fig.savefig("../../figures/" + fig_name + ".png")
    s.write_file(PSNP_figs + fig_name + ".eps", fig) # saves to Sciencedata 
    s.write_file(PSNP_figs + fig_name + ".png", fig)

In [32]:
file_data = s.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict")
credentials = service_account.Credentials.from_service_account_info(file_data)
gc = gspread.Client(auth=credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']))
socdiv_results = gc.open_by_url("https://docs.google.com/spreadsheets/d/1kACGBS9Xdno7FL2fPZ-vCOu4SSh9uUaZmGvnEghVs_A/edit?usp=sharing")

## Import the epigraphic dataset

In [33]:
# find local version of the input dataset
local_paths = !find ~/Github -name "LIST_v1-0.parquet"
print(local_paths)

['/home/au648560/Github/latin_socdiv/data/large_data/LIST_v1-0.parquet']


In [34]:
# read LIST dataset from zenodo or locally, if already available
try:
    LIST = gpd.read_parquet(local_paths[0])
    print("local copy found and loaded")
except:# !!! use carefully - takes time
    q = input("local file not found, try another put, or input 'y' for download")
    if q == "y":
        LIST = gpd.read_file("https://zenodo.org/record/8431323/files/LIST_v1-0.geojson?download=1", driver="geoJSON")
        print("file downloaded from zenodo")
        LIST.to_parquet("../../data/large_data/LISTg.parquet")

local copy found and loaded


In [35]:
print(LIST.columns.tolist())

['LIST-ID', 'EDCS-ID', 'EDH-ID', 'trismegistos_uri', 'pleiades_id', 'transcription', 'inscription', 'clean_text_conservative', 'clean_text_interpretive_sentence', 'clean_text_interpretive_word', 'clean_text_interpretive_word_EDCS', 'diplomatic_text', 'province', 'place', 'inscr_type', 'status_notation', 'inscr_process', 'status', 'partner_link', 'last_update', 'letter_size', 'type_of_inscription', 'work_status', 'year_of_find', 'present_location', 'text_edition', 'support_objecttype', 'support_material', 'support_decoration', 'keywords_term', 'people', 'type_of_inscription_clean', 'type_of_inscription_certainty', 'height_cm', 'width_cm', 'depth_cm', 'material_clean', 'type_of_monument_clean', 'type_of_monument_certainty', 'province_label_clean', 'province_label_certainty', 'country_clean', 'country_certainty', 'findspot_ancient_clean', 'findspot_ancient_certainty', 'modern_region_clean', 'modern_region_certainty', 'findspot_modern_clean', 'findspot_modern_certainty', 'findspot_clean', 


# Cities and population estimates of the western Roman provinces

In [36]:
roman_cities = gpd.read_file("../../data/roman_cities_pop.geojson")
pleiades_regions = gpd.read_file("../../data/pleiades_regions.geojson")
pleiades_regions_west = gpd.read_file("../../data/pleiades_regions_west.geojson")

In [37]:
# total population estimate based on calculations from Hanson and Ortman 2017
roman_cities["pop_est"].sum()

10214337.0

In [38]:
# how many cities have the population estimate larger than 30,000
(roman_cities["pop_est"] >= 30000).sum()

68

In [39]:
# how many cities belong to the western part of the Roman Empire (defined by the Jirecek line, running in the mid-Balkans)
roman_cities["western_province"].sum()

KeyError: 'western_province'

In [40]:
# proportion of all Hanson's cities belonging to the western empire
roman_cities["western_province"].sum() / len(roman_cities)

KeyError: 'western_province'

In [ ]:
# population estimate for the western empire, based on Hanson and Ortman's 2017 formula
roman_cities[roman_cities["western_province"]]["pop_est"].sum()

 # Latin inscriptions in the western provinces

In [41]:
regions_west_union = pleiades_regions_west.unary_union

In [42]:
# total size of the dataset, including the Roman East
len(LIST)

525870

In [43]:
LIST["is_western"] = LIST.within(regions_west_union)

KeyboardInterrupt: 

In [ ]:
# how many inscriptions come from the Roman West
LIST["is_western"].sum()

In [ ]:
# proportion of inscriptions belonging to the western part of the empire
LIST["is_western"].mean()

In [ ]:
fig, ax = plt.subplots(figsize=[5.2, 3.2])
pleiades_regions.plot(ax=ax, color="lightgray")
pleiades_regions_west.plot(ax=ax, color="darkgray")
#roman_cities[roman_cities["western_province"]].plot(ax=ax, color="green", markersize=0.5)
LIST[LIST["is_western"]].plot(ax=ax, color="darkblue", markersize=0.1)
ax.set_axis_off()
ax.set_xlim(-10, 41)
ax.set_ylim(23, 55.5)
plt.show()

In [ ]:
save_figure(fig, "LIST_western_spatial") 


 # Compute distance to settlement based on the size of its population estimate

In [ ]:
# defines the size of urban context based on the population estimates from Hansen and Ortman 2017


masks_dict = {}
masks_dict["large"] = roman_cities["pop_est"] >= 30000
masks_dict["medium"] = (roman_cities["pop_est"] >= 5000) & (roman_cities["pop_est"] < 30000)
masks_dict["small"] = small_cities_mask = roman_cities["pop_est"] < 5000

In [ ]:
# overview: how many cities per rank and their population estimates

for mask in masks_dict.keys():
    print(mask, len(roman_cities[masks_dict[mask]]), roman_cities[masks_dict[mask]]["pop_est"].sum())

In [ ]:
# custom function assigning each inscription the nearest city

geod = Geod(ellps="WGS84")
def get_dist_m(inscr_geo, city_geo):
    line_string = LineString([inscr_geo, city_geo])
    return np.round(geod.geometry_length(line_string), 2)

def ckdnearest(inscriptions, cities, city_type):
    cities = cities[masks_dict[city_type]]
    cities = cities[['ID', 'geometry']]
    cities.rename(columns={"ID" : city_type + "_city_ID"}, inplace=True)
    insr_geometries = np.array(list(inscriptions.geometry.apply(lambda x: (x.x, x.y))))
    cities_geometries = np.array(list(cities.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(cities_geometries)
    dist, idx = btree.query(insr_geometries, k=1)
    nearest_cities_df = cities.iloc[idx].rename(columns={"geometry": city_type +"_city_geo"}).reset_index(drop=True)
    nearest_cities_df[city_type + "_city_geo"] = [[geom.x, geom.y] for geom in nearest_cities_df[city_type + "_city_geo"]]

    merge_gdf = pd.concat(
        [
            inscriptions.reset_index(drop=True),
            nearest_cities_df,
            pd.Series(dist, name=city_type + "_city_dist")
        ],
        axis=1)
    merge_gdf[city_type + "_city_dist"] = merge_gdf.apply(lambda row: get_dist_m(row.geometry, row[city_type + "_city_geo"]), axis=1)
    return merge_gdf

In [46]:
LIST

,LIST-ID,EDCS-ID,EDH-ID,trismegistos_uri,pleiades_id,transcription,inscription,clean_text_conservative,clean_text_interpretive_sentence,clean_text_interpretive_word,clean_text_interpretive_word_EDCS,diplomatic_text,province,place,inscr_type,status_notation,inscr_process,status,partner_link,last_update,letter_size,type_of_inscription,work_status,year_of_find,present_location,text_edition,support_objecttype,support_material,support_decoration,keywords_term,people,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,language_EDCS,raw_dating,not_after,not_before,Longitude,Latitude,is_geotemporal,geometry,is_within_RE,urban_context,urban_context_city,urban_context_pop_est,type_of_inscription_auto,type_of_inscription_auto_prob
445463,445464,EDCS-24900077,HD056163,https://www.trismegistos.org/text/177366,570485,Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...,Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...,Q Caecilio C f Metelo imperatori Italici quei ...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metelo imperatori It...,Quinto Caecilio Cai filio Metello imperatori I...,Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...,Achaia,Agia Triada / Merbaka / Midea,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-11-11,None,honorific inscription,no image,NaN,None,\n Quinto Caecilio Cai filio Metelo imperatori...,None,None,1000,69,"[{'age: days': None, 'age: hours': None, 'age:...",honorific inscription,False,NaN,NaN,NaN,NULL,NULL,False,Achaia,False,Greece,False,Midea,False,Pelopónissos,False,Midhéa,False,NULL,False,None,-68 to -68,-68.0,-68.0,22.841200,37.649800,True,POINT (22.841 37.650),True,rural,None,NaN,honorific inscription,1.000000
445464,445465,EDCS-03700724,HD052964,https://www.trismegistos.org/text/121715,531064,Fortissimo et piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et piissimo Caesari domino nostro G...,Fortissimo et Piissimo Caesari domino nostro G...,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,Achaia,Agios Athanasios / Photike,tituli honorarii,"Augusti/Augustae, ordo equester, tria nomina",litterae erasae,Augusti/Augustae; litterae erasae; ordo eque...,http://db.edcs.eu/epigr/partner.php?s_language...,2014-09-16,3-5.3 cm,honorific inscription,checked with photo,NaN,Fragma Kalama,\n Fortissimo et piissimo Caesari domino nostr...,57,None,1000,69,"[{'age: days': None, 'age: hours': None, 'age:...",honorific inscription,False,99.0,67.0,67.0,NULL,statue base,False,Epirus,False,Greece,False,Photike,False,Ípeiros,False,Paramythía,False,{Agios Athanasios},False,None,309 to 313,313.0,309.0,20.766800,39.451200,True,POINT (20.767 39.451),True,rural,None,NaN,honorific inscription,1.000000
445465,445466,EDCS-13800065,HD017714,https://www.trismegistos.org/text/177100,570049,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei / quei Aegei negotiantur / P(ublium) ...,Italicei quei Aegei negotiantur P Rutilium P f...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,Italicei quei Aegei negotiantur Publium Rutili...,ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...,Achaia,Aigio / Egio / Aiyion / Aegeum,tituli honorarii,"officium/professio, ordo senatorius, tria nomi...",None,officium/professio; ordo senatorius; tituli ...,http://db.edcs.eu/epigr/partner.php?s_language...,2011-03-29,3.5-3.7 cm,votive inscription,checked with photo

In [55]:
# filter dataset to contain only inscriptions with geometry / NEEDS FIXING!!!
#LIST
#LISTgeo = gpd.GeoDataFrame(LIST, geometry=gpd.points_from_xy(LIST["geometry"].apply(lambda x: x[0]), LIST["geometry"].apply(lambda x: x[1])), crs={"init": "epsg:3857"})

no_lat_or_lon = LIST["Latitude" == None or "Longitude" == None] 
ids_no_lat_lon = no_lat_or_lon["LIST-ID"]
for row in LIST:
    if row["LIST-ID"] in no_lat_or_lon:
        LIST.drop(labels = row.index, axis = 0, in_place = True)
#GEO.dropna(inplace = True)
#LIST["geometry"] = GEO


KeyError: False

In [ ]:
%%time
LISTgeo = ckdnearest(LIST, roman_cities, "large")
LISTgeo = ckdnearest(LIST, roman_cities, "medium")
LISTgeo = ckdnearest(LIST, roman_cities, "small")

In [ ]:
LIST.head(5)

In [ ]:
fig, [ax1, ax2, ax3]=plt.subplots(3, 1, figsize=(5.2, 3.2), tight_layout=True)

for ax, label, color in zip([ax1, ax2, ax3], ["large", "medium", "small"], ["blue", "green", "red"]):
    sns.kdeplot(label + "_city_dist", color=color, data=LIST, ax=ax)
    ax.set_title(label)
    ax.set_xlim(0, 1000000)
    ax.set_xticks([0, 250000, 500000, 750000, 1000000])
    ax.set_xticklabels(["0", "250km", "500km", "750km", "1000km"])
    ax.set_yticklabels([])
    if label in ["large", "medium"]:
        ax.set_xticklabels([])
        ax.set_xlabel("")
    else:
        ax.set_xlabel("distance to nearest city of given type")

In [ ]:
save_figure(fig, "dist_to_nearest_by_type")

# Urban contexts and clustering of inscriptions

In [ ]:
# Bodel claims that Roman inscriptions are dispersed over an area of about 30 square km. Bodel 2001?
np.sqrt(30 / np.pi)

In [ ]:
# custom function creating a 5km buffer zone around each settlement/urban context


def get_urban_context(row):
    buffer=5000
    urban_context = "rural"
    if row["large_city_dist"] <= buffer:
        urban_context = "large"
    else:
        if row["medium_city_dist"] <= buffer:
            urban_context = "medium"
        else:
            if row["small_city_dist"] <= buffer:
                urban_context = "small"
    return urban_context



In [ ]:
# assigning inscriptions to the urban context within their 5 km buffer zone
urban_contexts_alt = LIST.apply(lambda row: get_urban_context(row), axis=1)

Original results, with big buffers: From the 136,190 inscriptions in the LIRE dataset, 48,799 have been mapped on a large city, 37,734 inscriptions on a medium city, and 26,836 inscriptions on a small city. The remaining 22,866 inscriptions have been classified as rural, since they are located out of the buffers of all cities.


In [ ]:
pd.DataFrame(pd.DataFrame(urban_contexts_alt).groupby(0).size())

In [ ]:
LIST["urban_context_alt"] = urban_contexts_alt

In [ ]:
LIST.to_parquet("../data/large_data/LIST_contexts.parquet")